# Basic usage of the train/test/finetune utilities

In [1]:
from physioex.train.utils import train, test, finetune
from physioex.train.models import load_model

from physioex.data import PhysioExDataModule

import os

In [2]:
# Note the best practice is to use a yaml file to store the arguments
# datamodule arguments

datamodule_kwargs = {
    "selected_channels": ["EEG"],
    "sequence_length": 3,
    "target_transform": None, # needs to match model output
    "preprocessing": "xsleepnet", # needs to match model input
    "data_folder": "/mnt/guido-data/", # your custom path here
    "num_workers": os.cpu_count(), # default value
}

# let's use the standard approach to load the model, can be any SleepModule
# you can use it also in a .yaml file

#model_class = "physioex.train.networks.seqsleepnet:SeqSleepNet"
model_class = "physioex.train.networks.wrapper:Wrapper"

import importlib
module_name, class_name = model_class.split(":")
model_class = getattr(importlib.import_module(module_name), class_name)

# now we need to define the model parameters, 
# these should be passed in the constructor of the model as a dictionary "model_config"

# first read the default parameters from physioex
from physioex.train.networks import config as networks_config
model_config = networks_config["default"]["model_kwargs"].copy()

# now we can modify the parameters as we want, first using the specific parameters for the model
if "model_kwargs" in networks_config["seqsleepnet"]:
    model_config.update(networks_config["seqsleepnet"]["model_kwargs"])

# now setup the ones that depends on the datamodule
model_config.update({
    "in_channels" : len(datamodule_kwargs["selected_channels"]),
    "sequence_length" : datamodule_kwargs["sequence_length"],
})

train_kwargs = {
    "datasets": "mass", # dataset to use, can be also a PhysioExDataModule
    "datamodule_kwargs": datamodule_kwargs,
    "model_class": model_class,
    "model_config": model_config,
    "batch_size": 128,
    "num_validations": 1, # number of validations to perform per epoch
    "checkpoint_path": "../models/basic_usage/", # path to save the model
    "max_epochs": 15,
    "resume": False, # if the train sees a checkpoint it will resume from there
    "checkpoint_metric": "val_loss", # metric to use to save the best model
    "checkpoint_metric_mode": "min", # mode to use to save the best model
}

In [3]:
# now we can train the model

best_checkpoint = train( **train_kwargs )

best_checkpoint = os.path.join( train_kwargs["checkpoint_path"], best_checkpoint )

Seed set to 42


Selected fold for dataset 0: 0
Selected fold for dataset 0: 0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/manni/miniconda3/envs/physioex/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /home/manni/physioex/models/basic_usage exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name | Type                 | Params | Mode 
-------------------------------------------------------
0  | nn   | Net                  | 193 K  | train
1  | wacc | MulticlassAccuracy   | 0      | train
2  | macc | MulticlassAccuracy   | 0      | train
3  | wf1  | MulticlassF1Score    | 0      | train
4  | mf1  | MulticlassF1Score    | 0      | train
5  | ck   | MulticlassCohenKappa | 0      | train
6  | pr   | MulticlassPrecision  | 0      | train
7  | rc   | MulticlassRecall     | 0      | train
8  | loss | CrossEntropyLoss     | 0      | train
9  | acc  | MulticlassAccuracy   | 0      | train
10 | Mf1  | MulticlassF1Score 

cuda:0


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/home/manni/miniconda3/envs/physioex/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [5]:
# now we should (optional) load and test the model on the test split of the dataset

# in case you want to load the model you can just use physioex.train.models:load_model

model = load_model(
    model = model_class,
    model_kwargs = model_config,
    ckpt_path = best_checkpoint,
)

# or you can use the API offered by the test utility

test_kwargs = {
    "datasets": "mass",
    "datamodule_kwargs": datamodule_kwargs,
    "model": None,  # if you want to use the model loaded before you can pass it here
    "model_class": model_class,
    "model_config": model_config,
    "batch_size": 128,
    "checkpoint_path": best_checkpoint,
    "results_path": "../models/basic_usage/"
}

hmc_results = test( **test_kwargs )

hmc_results.head() 
# note: fold:-1 is a random fold setted up at the beginning. HMC has 1 fold so it's always the '0'fold

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


cpu
Selected fold for dataset 0: 0
Selected fold for dataset 0: 0
cpu


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_Mf1          │     0.340486615896225     │
│         test_cel          │    1.2772881984710693     │
│          test_f1          │    0.3952943980693817     │
│         test_loss         │    18.550464630126953     │
│         test_mse          │    0.7239184975624084     │
│          test_r1          │     3.527681589126587     │
│          test_r2          │    1.1086255311965942     │
│       test_rec_loss       │    0.8569663166999817     │
│       test_std_pen        │    0.21156243979930878    │
│      test_std_pen_F       │    0.19471769034862518    │
│      test_std_pen_T       │    0.3080638647079468     │
└───────────────────────────┴───────────────────────────┘

,test_loss,test_cel,test_r1,test_r2,test_rec_loss,test_mse,test_std_pen,test_std_pen_T,test_std_pen_F,test_f1,test_Mf1,dataset,fold
0,18.550465,1.277288,3.527682,1.108626,0.856966,0.723918,0.211562,0.308064,0.194718,0.395294,0.340487,hmc,-1


In [4]:
# now we can finetune the model on the dcsm dataset

# first let's see the accuracy of the model on the dcsm dataset
test_kwargs["datasets"] = "mass"
dcsm_results = test( **test_kwargs )

dcsm_results.head()
# note : the same result could be obtained setting datasets as ["hmc", "dcsm"]

NameError: name 'test_kwargs' is not defined

In [20]:

# now finetune the model on the dcsm dataset decreasing the learning rate

train_kwargs["datasets"] = "dcsm"
train_kwargs["checkpoint_path"] = "../models/basic_usage/finetuned/"

finetune_kwargs = {
    "model": None,
    "model_class": model_class,
    "model_config": model_config,
    "model_checkpoint": best_checkpoint,
    "learning_rate": 1e-7,
    "train_kwargs": train_kwargs,
}

new_best_checkpoint = finetune( **finetune_kwargs )

new_best_checkpoint = os.path.join( train_kwargs["checkpoint_path"], new_best_checkpoint )


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/guido/miniconda3/envs/physioex/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/guido/github/physioex-private/models/basic_usage/finetuned exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                 | Params | Mode 
------------------------------------------------------
0 | nn   | Net                  | 137 K  | train
1 | wacc | MulticlassAccuracy   | 0      | train
2 | macc | MulticlassAccuracy   | 0      | train
3 | wf1  | MulticlassF1Score    | 0      | train
4 | mf1  | MulticlassF1Score    | 0      | train
5 | ck   | MulticlassCohenKappa | 0      | train
6 | pr   | MulticlassPrecision  | 0      | train
7 | rc   | MulticlassRecall     | 0      | train
8 | loss | CrossEntropyLoss     | 0      | train
---------------------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [24]:
# now we can test the model on both the datasets

test_kwargs["checkpoint_path"] = new_best_checkpoint
test_kwargs["results_path"] = "../models/basic_usage/finetuned/"
test_kwargs["datasets"] = ["hmc", "dcsm"]
test_kwargs["aggregate_datasets"] = False # we want to have results for each dataset not for the aggregation of them

finetuned_results = test( **test_kwargs )

finetuned_results.head()


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7087806463241577     │
│          test_ck          │    0.6214028000831604     │
│          test_f1          │    0.7025613784790039     │
│         test_loss         │    0.7353542447090149     │
│         test_macc         │    0.6813777089118958     │
│         test_mf1          │    0.6640945076942444     │
│          test_pr          │    0.7089968919754028     │
│          test_rc          │    0.7087806463241577     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5887628793716431     │
│          test_ck          │    0.4236217141151428     │
│          test_f1          │    0.6168046593666077     │
│         test_loss         │    1.4943362474441528     │
│         test_macc         │     0.605950653553009     │
│         test_mf1          │    0.5350980758666992     │
│          test_pr          │    0.7684549689292908     │
│          test_rc          │    0.5887628793716431     │
└───────────────────────────┴───────────────────────────┘

,test_loss,test_acc,test_f1,test_ck,test_pr,test_rc,test_macc,test_mf1,dataset,fold
0,0.735354,0.708781,0.702561,0.621403,0.708997,0.708781,0.681378,0.664095,hmc,-1
1,1.494336,0.588763,0.616805,0.423622,0.768455,0.588763,0.605951,0.535098,dcsm,-1
